In [3]:
import os
import joblib

import tensorflow as tf
from tensorflow import feature_column
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, Model

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

2024-05-22 11:51:28.923513: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
df = pd.read_csv("/home/riri/Desktop/Soiling/datasets/soiling.csv")
df.head()

,client,gender,age,hours_slept,sleep_quality,main_food_taken,food_taken_amount,drink,temperature(deg),exercise,medication,breakfast_time,lunch_time,supper_time,visit_restroom,times_visited,avg_relieve_time(min),soiled
0,Fox,Female,8,8,Excellent,Ugali,Normal,Porridge,23.8,Yes,Yes,08:14,13:30,19:31,Yes,3,16,No
1,Charlton,Female,7,8,Excellent,Chapati,Small,Milk,23.2,No,Yes,08:53,12:32,18:28,No,0,18,No
2,Iqbal,Male,7,7,Excellent,Meat,Heavy,Porridge,18.7,No,Yes,09:43,13:22,19:23,No,0,19,No
3,Marc,Female,10,7,Excellent,Snacks,Normal,Coffee,18.2,No,No,09:46,13:16,19:32,No,0,22,Yes
4,King,Female,14,6,Good,Veggies,Normal,Water,20.1,Yes,No,08:57,13:55,18:05,Yes,4,18,No


In [5]:
df['target'] = np.where(df['soiled'] == 'Yes', 1, 0)
df.head()

,client,gender,age,hours_slept,sleep_quality,main_food_taken,food_taken_amount,drink,temperature(deg),exercise,medication,breakfast_time,lunch_time,supper_time,visit_restroom,times_visited,avg_relieve_time(min),soiled,target
0,Fox,Female,8,8,Excellent,Ugali,Normal,Porridge,23.8,Yes,Yes,08:14,13:30,19:31,Yes,3,16,No,0
1,Charlton,Female,7,8,Excellent,Chapati,Small,Milk,23.2,No,Yes,08:53,12:32,18:28,No,0,18,No,0
2,Iqbal,Male,7,7,Excellent,Meat,Heavy,Porridge,18.7,No,Yes,09:43,13:22,19:23,No,0,19,No,0
3,Marc,Female,10,7,Excellent,Snacks,Normal,Coffee,18.2,No,No,09:46,13:16,19:32,No,0,22,Yes,1
4,King,Female,14,6,Good,Veggies,Normal,Water,20.1,Yes,No,08:57,13:55,18:05,Yes,4,18,No,0


In [6]:
df.drop(columns=['client', 'breakfast_time', 'lunch_time', 'supper_time', 'soiled'], inplace=True)

In [7]:
train, test = train_test_split(df, test_size=0.2)
train, valid = train_test_split(train, test_size=0.2)
print(len(train), "training examples")
print(len(test), "test examples")
print(len(valid), "validation examples")

18260 training examples
5707 test examples
4565 validation examples


In [8]:
def transform_df(dataframe: pd.DataFrame, shuffle: bool=True, batch_size: int=32):
    df = dataframe.copy()
    lables = df.pop('target')
    ds = tf.data.Dataset.from_tensor_slices((dict(df), lables))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(df))
    ds.batch(batch_size)
    
    return ds

In [9]:
train_ds = transform_df(train)
val_ds = transform_df(valid, shuffle=False)
test_ds = transform_df(test, shuffle=False)

In [10]:
for feature_batch, lable_batch in train_ds.take(1):
    print("All features:", list(feature_batch.keys()))
    print("Batch of ages:", feature_batch['age'])
    print("A batch of targets:", lable_batch.numpy())

All features: ['gender', 'age', 'hours_slept', 'sleep_quality', 'main_food_taken', 'food_taken_amount', 'drink', 'temperature(deg)', 'exercise', 'medication', 'visit_restroom', 'times_visited', 'avg_relieve_time(min)']
Batch of ages: tf.Tensor(11, shape=(), dtype=int64)
A batch of targets: 1


2024-05-22 11:52:49.673618: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [11]:
example_batch = next(iter(train_ds))[0]

In [12]:
def demo(feature_column):
    feature_layer = layers.DenseFeatures(feature_column)
    print(feature_layer(example_batch).numpy())

In [14]:
tf.keras.layers.DenseFeatures

AttributeError: module 'keras._tf_keras.keras.layers' has no attribute 'DenseFeatures'

### Numeric cols.

In [12]:
temp = feature_column.numeric_column('hours_slept')
demo(temp)

Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.


NameError: name 'demo' is not defined